In [3]:
# from transformers import AutoModelForCausalLM, AutoTokenizer

# model_name = "Qwen/Qwen2.5-7B-Instruct"

# model = AutoModelForCausalLM.from_pretrained(
#     model_name,
#     torch_dtype="auto",
#     device_map="auto"
# )
# tokenizer = AutoTokenizer.from_pretrained(model_name)

# prompt = "Give me a short introduction to large language model."
# messages = [
#     {"role": "system", "content": "You are Qwen, created by Alibaba Cloud. You are a helpful assistant."},
#     {"role": "user", "content": prompt}
# ]
# text = tokenizer.apply_chat_template(
#     messages,
#     tokenize=False,
#     add_generation_prompt=True
# )
# model_inputs = tokenizer([text], return_tensors="pt").to(model.device)

# generated_ids = model.generate(
#     **model_inputs,
#     max_new_tokens=512
# )
# generated_ids = [
#     output_ids[len(input_ids):] for input_ids, output_ids in zip(model_inputs.input_ids, generated_ids)
# ]

# response = tokenizer.batch_decode(generated_ids, skip_special_tokens=True)[0]


In [42]:
file_id = 1
answer_df_path = "../../qwen_data/answer_df_part"+str(file_id)+".csv"

In [35]:
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer


print(torch.cuda.is_available())
if torch.cuda.is_available():
    torch.cuda.empty_cache()
device = 0 if torch.cuda.is_available() else -1  # Use GPU if available, otherwise fallback to CPU

model_name = "Qwen/Qwen2.5-7B-Instruct"

model = AutoModelForCausalLM.from_pretrained(
    model_name,
    torch_dtype=torch.float16, #torch_dtype="auto",
    device_map="auto" if device == 0 else None,  # Automatically distribute across devices or use CPU
)
tokenizer = AutoTokenizer.from_pretrained(model_name)


True


Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

In [43]:
import pandas as pd
import numpy as np
answer_df = pd.read_csv(answer_df_path)
query_df = pd.read_csv("../../data/fea_df.csv")

if not 'answer_string' in answer_df.columns:
    answer_df['answer_string'] = np.nan
    
print(answer_df.shape)
print(answer_df.iloc[175:185,])

(9647, 3)
       sm_id                                         text_w_eos  \
175  1062gua  have gotten into fryingbaking super firm tofu ...   
176  1066t0q  anyone else go through cycles of feeling attra...   
177  1068zm3  im such a picky eater and its so formal sit do...   
178  106gk9g  this is the thread for week ly questions and s...   
179  106hm9e  i just want to know if there is any difference...   
180  106m5hx  full texthttpswww. sciencedirect. comsciencear...   
181  106pbcy  was in santa fe for christmas and i am missing...   
182  106qdg2  i am struggling with severe fatigue in the mor...   
183  106sguj  i sometimes add salt when i am cooking and i a...   
184  106t0bv  i am an italian student and i want to eat heal...   

                                         answer_string  
175  (1) relation: no,\n(2) protein: yes, related p...  
176  (1) relation: no,\n(2) protein: no,\n(3) ed: n...  
177  (1) relation: no,\n(2) protein: no,\n(3) ed: n...  
178  (1) relation: no,\n

In [44]:
system_message = """
You are an AI assistant designed to answer questions.
Please restrict your answer to the exact question and use the exact answer format asked.
Answer should not have implied information. If the answer is yes, always provide related phrases. 
"""

def format_user_message(text):
    question_content = "Does the paragraph mention any of the following topics:\n"
    for i in range(len(query_df)):
        question_content += f"  ({i+1}) {query_df.fea[i]}: {query_df.description[i]}.\n"
    answer_content = "Return answer in format:\n"
    for i in range(len(query_df)): 
        answer_content += f"  ({i+1}) {query_df.fea[i]}: [yes/no], related phrases if any: \n"
    paragragh_content = f"Paragraph: '{text}' \n"
    user_message = question_content + answer_content + paragragh_content
    #print(user_message)
    
    return user_message




In [ ]:
from tqdm import tqdm
for k in tqdm(range(1000), desc="Processing batch"):
    batch_size = 10

    # Filter for rows where 'answer_string' is NaN
    unanswered_df = answer_df[answer_df['answer_string'].isna()]

    # Get the indices of these NaN entries in the original DataFrame
    indices_to_update = unanswered_df.index[:batch_size]

    # Prepare prompt content for the first 10 entries with NaN answer_string
    user_messages = [format_user_message(text) for text in unanswered_df['text_w_eos'].iloc[:batch_size]]

    # Save the indices list if needed for later use
    indices_to_update_list = list(indices_to_update)

    # query process
    messages = [ [ {"role": "system", "content": system_message}, {"role": "user", "content": user_message}] for user_message in user_messages]
    inputs = [tokenizer.apply_chat_template(
        message,
        tokenize=False,
        add_generation_prompt=True ) for message in messages]
    responses = []
    for i in range(len(inputs)):
        model_inputs = tokenizer([inputs[i]], return_tensors="pt").to(model.device)
        generated_ids = model.generate(
            **model_inputs,
            max_new_tokens=512,
            temperature=0.1
        )
        generated_ids = [
            output_ids[len(input_ids):] for input_ids, output_ids in zip(model_inputs.input_ids, generated_ids)
        ]
        responses.append(tokenizer.batch_decode(generated_ids, skip_special_tokens=True)[0])
    
    # Display and process responses in a loop
    for i, response in enumerate(responses):
        #display(Markdown(colorize_text(f"{response}")))
        # Extract answer if available
        answer = response
        # Use the original index from indices_to_update_list
        answer_df.loc[indices_to_update_list[i], 'answer_string'] = answer

    print(answer_df.loc[indices_to_update_list, ['sm_id','answer_string']])
    answer_df.to_csv(answer_df_path, index=False)
    


Processing batch:   0%|          | 1/1000 [00:59<16:27:48, 59.33s/it]

       sm_id                                      answer_string
180  106m5hx  (1) relation: no,\nrelated phrases if any: \n(...
181  106pbcy  (1) relation: no,\n(2) protein: no,\n(3) ed: n...
182  106qdg2  (1) relation: no,\n(2) protein: no,\n(3) ed: n...
183  106sguj  (1) relation: no,\n(2) protein: no,\n(3) ed: n...
184  106t0bv  (1) relation: no,\nrelated phrases if any: \n(...
185  106tan5  (1) relation: no,\n(2) protein: no,\n(3) ed: n...
186  106u3dy  (1) relation: no,\n(2) protein: no,\n(3) ed: n...
187  106wc2c  (1) relation: no,\n(2) protein: no,\n(3) ed: n...
188  106xam9  (1) relation: no,\n(2) protein: yes, related p...
189  1071kgl  (1) relation: no,\n(2) protein: no,\n(3) ed: n...


Processing batch:   0%|          | 2/1000 [01:55<15:58:01, 57.60s/it]

       sm_id                                      answer_string
190  1071q1y  (1) relation: no,\n(2) protein: no,\n(3) ed: n...
191  107239r  (1) relation: no,\n(2) protein: no,\n(3) ed: n...
192  1072c8b  (1) relation: no,\n(2) protein: no,\n(3) ed: y...
193  1073fp2  (1) relation: no,\n(2) protein: no,\n(3) ed: n...
194  1075g7b  (1) relation: no\n(2) protein: no\n(3) ed: no\...
195  1075tqt  (1) relation: no\n(2) protein: no\n(3) ed: no\...
196  1078sex  (1) relation: no,\n(2) protein: no,\n(3) ed: n...
197  107c8gm  (1) relation: no,\n(2) protein: yes, related p...
198  107cccb  (1) relation: no,\n(2) protein: no,\n(3) ed: n...
199  107etzd  (1) relation: no,\nrelated phrases if any: \n(...


Processing batch:   0%|          | 3/1000 [02:48<15:21:52, 55.48s/it]

...

        sm_id                                      answer_string
1560  11n0ope  (1) relation: no,\n(2) protein: no,\n(3) ed: n...
1561  11n3pwz  (1) relation: no\n(2) protein: no\n(3) ed: no\...
1562  11n5le9  (1) relation: no,\n(2) protein: no,\n(3) ed: n...
1563  11n618r  (1) relation: no\n(2) protein: no\n(3) ed: yes...
1564  11n681p  (1) relation: no,\n(2) protein: no,\n(3) ed: n...
1565  11n8byi  (1) relation: no\n(2) protein: no\n(3) ed: no\...
1566  11n8giv  (1) relation: no\n(2) protein: no\n(3) ed: no\...
1567  11n8w85  (1) relation: no,\nrelated phrases if any: \n(...
1568  11ncedx  (1) relation: no,\n(2) protein: no,\n(3) ed: n...
1569  11nchb3  (1) relation: no,\n(2) protein: no,\n(3) ed: n...


Processing batch:  14%|█▍        | 140/1000 [2:15:35<14:02:14, 58.76s/it]

        sm_id                                      answer_string
1570  11ncy72  (1) relation: no,\n(2) protein: no,\n(3) ed: y...
1571  11nfryv  (1) relation: no\n(2) protein: no\n(3) ed: no\...
1572  11ngtva  (1) relation: no,\n(2) protein: no,\n(3) ed: n...
1573  11njss7  (1) relation: no,\n(2) protein: no,\n(3) ed: y...
1574  11nm10j  (1) relation: no,\n(2) protein: no,\n(3) ed: n...
1575  11npnre  (1) relation: no,\n(2) protein: no,\n(3) ed: n...
1576  11nptwa  (1) relation: no,\n(2) protein: no,\n(3) ed: n...
1577  11nsus0  (1) relation: no,\n(2) protein: no,\n(3) ed: n...
1578  11ntbar  (1) relation: no\n(2) protein: yes, related ph...
1579  11nuj8p  (1) relation: no,\n(2) protein: no,\n(3) ed: n...


Processing batch:  14%|█▍        | 141/1000 [2:16:29<13:40:35, 57.32s/it]

        sm_id                                      answer_string
1580  11nvasm  (1) relation: no,\n(2) protein: no,\n(3) ed: n...
1581  11nvoli  (1) relation: no,\n(2) protein: no,\n(3) ed: n...
1582  11nx75r  (1) relation: no,\n(2) protein: no,\n(3) ed: n...
1583  11nxici  (1) relation: no\n(2) protein: no\n(3) ed: no\...
1584  11ny4x8  (1) relation: no\n(2) protein: no\n(3) ed: no\...
1585  11nz5xo  (1) relation: no,\n(2) protein: no,\n(3) ed: n...
1586  11o0iu3  (1) relation: no,\n(2) protein: no,\n(3) ed: n...
1587  11o0q30  (1) relation: no,\n(2) protein: no,\n(3) ed: n...
1588  11o0v44  (1) relation: no,\n(2) protein: no,\n(3) ed: n...
1589  11o1cn1  (1) relation: no,\n(2) protein: no,\n(3) ed: n...
